In [1]:
#!:bash
mkdir coco
cd coco
mkdir images
cd images

wget http://images.cocodataset.org/zips/train2017.zip
wget http://images.cocodataset.org/zips/val2017.zip
wget http://images.cocodataset.org/zips/test2017.zip
wget http://images.cocodataset.org/zips/unlabeled2017.zip

In [ ]:
#!:bash
cd coco
cd images

unzip train2017.zip
unzip val2017.zip
unzip test2017.zip
unzip unlabeled2017.zip

cd ../
wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
wget http://images.cocodataset.org/annotations/stuff_annotations_trainval2017.zip
wget http://images.cocodataset.org/annotations/image_info_test2017.zip
wget http://images.cocodataset.org/annotations/image_info_unlabeled2017.zip

unzip annotations_trainval2017.zip
unzip stuff_annotations_trainval2017.zip
unzip image_info_test2017.zip
unzip image_info_unlabeled2017.zip

In [3]:
#!:bash
wget -c https://s3-us-west-2.amazonaws.com/dl.fbaipublicfiles.com/LVIS/lvis_v1_val.json.zip
unzip lvis_v1_val.json.zip

In [4]:
#!:bash
wget -c https://s3-us-west-2.amazonaws.com/dl.fbaipublicfiles.com/LVIS/lvis_v1_train.json.zip
unzip lvis_v1_train.json.zip

In [72]:
#!L
import json
from tqdm import tqdm

In [75]:
#!L
def parse_coco(inp_file, out_file, syn=False):
    with open(inp_file, 'r') as f:
        annotations = json.loads(f.read())

    img = annotations['images']
    ann = annotations['annotations']
    cat = annotations['categories']
    del annotations

    img_bboxid  = dict()  # img: bboxes ids
    img_bbox = dict()  # img: bboxes
    img_name = dict()  # img: filename
    bboxid_bbox = dict()  # bboxid: bbox
    bboxid_catid = dict()  # bbox id: cat id 
    catid_catname = dict()  # cat id: cat name

    for annotation in tqdm(ann):
        if img_bboxid.get(int(annotation['image_id'])) is None:
            img_bboxid[int(annotation['image_id'])] = [int(annotation['id'])]
            img_bbox[int(annotation['image_id'])] = [annotation['bbox']]
        else:
            img_bboxid[int(annotation['image_id'])].append(int(annotation['id']))
            img_bbox[int(annotation['image_id'])].append(annotation['bbox'])
        bboxid_bbox[int(annotation['id'])] = annotation['bbox']
        bboxid_catid[int(annotation['id'])] = int(annotation['category_id'])

    for image in tqdm(img):
        img_name[int(image['id'])] = image['file_name'].split('/')[-1]

    for category in tqdm(cat):
        if syn:
            catid_catname[int(category['id'])] = category['synonyms']
        else:
            catid_catname[int(category['id'])] = category['name'] 

    with open(out_file, 'w') as f:
        f.write(json.dumps({
            'img_bboxid': img_bboxid, 
            'img_bbox': img_bbox, 
            'bboxid_bbox': bboxid_bbox, 
            'bboxid_catid': bboxid_catid,
            'img_name': img_name,
            'catid_catname': catid_catname
        }))

In [77]:
#!L
parse_coco('./pascal_trainval0712.json', './annotations_val_pascal0712.json')

In [71]:
#!L
parse_coco('./coco/annotations/instances_val2017.json', './annotations_val_coco.json')

In [65]:
parse_coco('./coco/annotations/instances_train2017.json', './annotations_train_coco.json')

In [66]:
parse_coco('./lvis_v1_val.json', './annotations_val_lvis.json', syn=True)

In [67]:
parse_coco('./lvis_v1_train.json', './annotations_train_lvis.json', syn=True)

In [46]:
import matplotlib.pyplot as plt
from PIL import Image

In [39]:
with open('./annotations_val_coco.json', 'r') as f:
    annotations = json.loads(f.read())

In [68]:
plt.imshow(Image.open(f"./coco/images/val2017/{annotations['img_name']['4134']}"))
bboxes = np.array(annotations['img_bbox']['4134'])
for x1, y1, x2, y2 in bboxes:
    plt.plot([x1, x1 + x2, x1 + x2, x1, x1], [y1, y1, y1 + y2, y1 + y2, y1])